In [86]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache , SQLiteCache

# set_llm_cache(InMemoryCache())  # 모든 cache data가 memory 안에 저장됨.
set_llm_cache(SQLiteCache("cache.db"))
set_debug(True) 

chat = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,
    # streaming=True,
    # callbacks=[StreamingStdOutCallbackHandler()]
)

chat.predict("이탈리아 파스타는 어떻게 만드나요?.")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 이탈리아 파스타는 어떻게 만드나요?."
  ]
}
[llm/end] [1:llm:ChatOpenAI] [3.44s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "이탈리아 파스타를 만드는 방법은 다양하지만, 전통적인 방법은 다음과 같습니다.\n\n1. 물에 소금을 넣고 끓는 물에 파스타를 넣어 삶습니다. 삶는 시간은 각 종류마다 다르므로 포장지에 적힌 시간을 참고하세요.\n\n2. 파스타를 삶는 동안, 별도의 팬에 올리브 오일을 데워 다양한 재료를 넣어 소스를 만듭니다. 예를 들어, 마늘, 양파, 토마토, 바질, 올리브, 마른 햄, 새우 등을 사용할 수 있습니다.\n\n3. 삶은 파스타를 물기를 제거하고 소스와 섞어줍니다. 소스가 파스타에 골고루 베이도록 잘 섞어주세요.\n\n4. 접시에 담아서 파마산 치즈나 파슬리를 곁들여 마무리합니다.\n\n이처럼 이탈리아 파스타는 간단하지만 다양한 소스와 재료를 활용하여 맛을 더할 수 있습니다. 원하는 재료와 소스를 활용하여 자신만의 파스타를 만들어보세요.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "cont

'이탈리아 파스타를 만드는 방법은 다양하지만, 전통적인 방법은 다음과 같습니다.\n\n1. 물에 소금을 넣고 끓는 물에 파스타를 넣어 삶습니다. 삶는 시간은 각 종류마다 다르므로 포장지에 적힌 시간을 참고하세요.\n\n2. 파스타를 삶는 동안, 별도의 팬에 올리브 오일을 데워 다양한 재료를 넣어 소스를 만듭니다. 예를 들어, 마늘, 양파, 토마토, 바질, 올리브, 마른 햄, 새우 등을 사용할 수 있습니다.\n\n3. 삶은 파스타를 물기를 제거하고 소스와 섞어줍니다. 소스가 파스타에 골고루 베이도록 잘 섞어주세요.\n\n4. 접시에 담아서 파마산 치즈나 파슬리를 곁들여 마무리합니다.\n\n이처럼 이탈리아 파스타는 간단하지만 다양한 소스와 재료를 활용하여 맛을 더할 수 있습니다. 원하는 재료와 소스를 활용하여 자신만의 파스타를 만들어보세요.'

In [87]:
chat.predict("이탈리아 파스타는 어떻게 만드나요?.") # 계속된 같은 답변에 캐싱된 데이터를 바로 전달!

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 이탈리아 파스타는 어떻게 만드나요?."
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "이탈리아 파스타를 만드는 방법은 다양하지만, 전통적인 방법은 다음과 같습니다.\n\n1. 물에 소금을 넣고 끓는 물에 파스타를 넣어 삶습니다. 삶는 시간은 각 종류마다 다르므로 포장지에 적힌 시간을 참고하세요.\n\n2. 파스타를 삶는 동안, 별도의 팬에 올리브 오일을 데워 다양한 재료를 넣어 소스를 만듭니다. 예를 들어, 마늘, 양파, 토마토, 바질, 올리브, 마른 햄, 새우 등을 사용할 수 있습니다.\n\n3. 삶은 파스타를 물기를 제거하고 소스와 섞어줍니다. 소스가 파스타에 골고루 베이도록 잘 섞어주세요.\n\n4. 접시에 담아서 파마산 치즈나 파슬리를 곁들여 마무리합니다.\n\n이처럼 이탈리아 파스타는 간단하지만 다양한 소스와 재료를 활용하여 맛을 더할 수 있습니다. 원하는 재료와 소스를 활용하여 자신만의 파스타를 만들어보세요.",
        "generation_info": {
          "finish_reason": "stop"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content":

'이탈리아 파스타를 만드는 방법은 다양하지만, 전통적인 방법은 다음과 같습니다.\n\n1. 물에 소금을 넣고 끓는 물에 파스타를 넣어 삶습니다. 삶는 시간은 각 종류마다 다르므로 포장지에 적힌 시간을 참고하세요.\n\n2. 파스타를 삶는 동안, 별도의 팬에 올리브 오일을 데워 다양한 재료를 넣어 소스를 만듭니다. 예를 들어, 마늘, 양파, 토마토, 바질, 올리브, 마른 햄, 새우 등을 사용할 수 있습니다.\n\n3. 삶은 파스타를 물기를 제거하고 소스와 섞어줍니다. 소스가 파스타에 골고루 베이도록 잘 섞어주세요.\n\n4. 접시에 담아서 파마산 치즈나 파슬리를 곁들여 마무리합니다.\n\n이처럼 이탈리아 파스타는 간단하지만 다양한 소스와 재료를 활용하여 맛을 더할 수 있습니다. 원하는 재료와 소스를 활용하여 자신만의 파스타를 만들어보세요.'